In [1]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))


Device: cuda:0


In [2]:
class MNISTReader(datasets.VisionDataset):
    def __init__(self, root: str) -> None:
        super().__init__(root)
        self.data_label = torch.load(root)
        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])

    def __len__(self) -> int:
        return len(self.data_label)

    def __getitem__(self, index):
        image, target = self.data_label[index]
        return self.transform(image), target

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 6, 5, 1, 2),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5, 1, 0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(16 * 5 * 5, 120),  
            nn.ReLU(True),
            nn.Linear(120, 84),
            nn.ReLU(True),
            nn.Linear(84, 2),
        )

    def forward(self, x):
        return self.model(x)


In [6]:
def train_one_model(train_path, epochs=20, batch_size=32, learning_rate=0.005):
    print(f"Loading training data from: {train_path}")
    train_dataset = MNISTReader(train_path)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    model = LeNet().to(device)
    loss_function = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        model.train()
        total_correct = 0
        total_samples = 0
        running_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            total_correct += (outputs.argmax(1) == targets).sum().item()
            total_samples += inputs.size(0)

        epoch_loss = running_loss / total_samples
        epoch_acc = total_correct / total_samples
        print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

    return model


In [7]:
def test_model(model, test_path, batch_size=1000):
    print(f"Testing on dataset: {test_path}")
    test_dataset = MNISTReader(test_path)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            total_correct += (outputs.argmax(1) == targets).sum().item()
            total_samples += inputs.size(0)

    accuracy = total_correct / total_samples
    print(f"Test accuracy: {accuracy:.4f}")
    return accuracy

In [9]:
train_files = [
    './data/ColoredMNIST/train1.pt',
    './data/ColoredMNIST/train2.pt',
    './data/ColoredMNIST/train3.pt',
]
# 测试集文件列表
test_files = [
    './data/ColoredMNIST/test1.pt',
    './data/ColoredMNIST/test2.pt',
]

results = {}

for train_path in train_files:
    print("="*50)
    print(f"Training model on {os.path.basename(train_path)}")
    model = train_one_model(train_path, epochs=20)

    # 测试模型在两个测试集上的表现
    accuracies = {}
    for test_path in test_files:
        acc = test_model(model, test_path)
        accuracies[os.path.basename(test_path)] = acc
    results[os.path.basename(train_path)] = accuracies

print("\nAll results:")
for train_set, test_accs in results.items():
    print(f"Model trained on {train_set}:")
    for test_set, acc in test_accs.items():
        print(f"  Test on {test_set}: Accuracy = {acc:.4f}")

Training model on train1.pt
Loading training data from: ./data/ColoredMNIST/train1.pt


C:\Users\ASUS\AppData\Local\Temp\ipykernel_27744\2010497841.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data_label = torch.load(root)


Epoch 1/20 - Loss: 0.2307, Accuracy: 0.8973
Epoch 2/20 - Loss: 0.0827, Accuracy: 0.9719
Epoch 3/20 - Loss: 0.0583, Accuracy: 0.9805
Epoch 4/20 - Loss: 0.0446, Accuracy: 0.9841
Epoch 5/20 - Loss: 0.0372, Accuracy: 0.9878
Epoch 6/20 - Loss: 0.0286, Accuracy: 0.9905
Epoch 7/20 - Loss: 0.0261, Accuracy: 0.9925
Epoch 8/20 - Loss: 0.0272, Accuracy: 0.9902
Epoch 9/20 - Loss: 0.0213, Accuracy: 0.9934
Epoch 10/20 - Loss: 0.0180, Accuracy: 0.9936
Epoch 11/20 - Loss: 0.0205, Accuracy: 0.9932
Epoch 12/20 - Loss: 0.0216, Accuracy: 0.9936
Epoch 13/20 - Loss: 0.0178, Accuracy: 0.9938
Epoch 14/20 - Loss: 0.0127, Accuracy: 0.9956
Epoch 15/20 - Loss: 0.0199, Accuracy: 0.9936
Epoch 16/20 - Loss: 0.0104, Accuracy: 0.9961
Epoch 17/20 - Loss: 0.0235, Accuracy: 0.9930
Epoch 18/20 - Loss: 0.0088, Accuracy: 0.9975
Epoch 19/20 - Loss: 0.0110, Accuracy: 0.9966
Epoch 20/20 - Loss: 0.0068, Accuracy: 0.9982
Testing on dataset: ./data/ColoredMNIST/test1.pt
Test accuracy: 0.6105
Testing on dataset: ./data/ColoredMNIS